In [11]:
import numpy as np
import pandas as pd
import yfinance as yf
import time, datetime
from arcticdb import Arctic, QueryBuilder
import yfinance as yf
import talib
# from arcticdb import Arctic, QueryBuilder
from utils import ac

lib = ac.get_library('stocks', create_if_missing=True)

symbol_df = pd.read_csv("us_stocks/symbols.csv",delimiter=',',index_col='Symbol').drop('Unnamed: 0',axis=1)
# Keep the first occurrence of each symbol
symbol_df = symbol_df[~symbol_df.index.duplicated(keep='first')]

symbols = symbol_df.index.unique().to_list()

In [13]:
len(symbols)

4606

In [23]:
def split_symbols(symbols, chunk_size):
    """Split a list of symbols into smaller chunks."""
    for i in range(0, len(symbols), chunk_size):
        yield symbols[i:i + chunk_size]

# Function to download and store data
def download_and_store(chunk):
    data = yf.download(chunk, group_by="Ticker", period="3y", auto_adjust=True)

    df = data.stack(level=0).rename_axis(['Date', 'Symbol']).reset_index(level=1)
    df = df.sort_values(by='Symbol',axis='index',kind='stable')
    print(df)
    # Add additional information
    df["Name"] = df["Symbol"].map(symbol_df["Name"])
    df['Sector'] = df['Symbol'].map(symbol_df['Sector'])

    df["20D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=20).mean().reset_index(level=0, drop=True)
    df["50D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=50).mean().reset_index(level=0, drop=True)
    df["200D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=200).mean().reset_index(level=0, drop=True)
    # df['ATR'] = df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=14)).reset_index(level=0, drop=True)
    print(df)
    def calculate_atr(group):
        print(group['Close'])
        return talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=20)

    df['ATR'] = df.groupby('Symbol').apply(calculate_atr).reset_index(level=0, drop=True)

    df['1M'] = df.groupby('Symbol')['Close'].pct_change(21)
    df['3M'] = df.groupby('Symbol')['Close'].pct_change(63)
    df['6M'] = df.groupby('Symbol')['Close'].pct_change(126)
    df['12M'] = df.groupby('Symbol')['Close'].pct_change(252)
    df['RS IBD'] = 2*df['3M']+df['6M']+df['12M'] # IBD Relative Strength =  2x 3M + 1x 6M + 1x 12M
    df['RS Rank'] = df.groupby(df.index)['RS IBD'].rank(pct=True)
    df["RS Rank 20D MA"] = df.groupby("Symbol")["RS Rank"].rolling(window=20).mean().reset_index(level=0, drop=True)

    for symbol in df.Symbol.unique().to_list():
        stock_data = df[df['Symbol']==symbol]
        # Store in Arctic
        print(f"saving {symbol} to arcticdb")
        lib.write(symbol, stock_data)



In [24]:
# Example usage
chunked_symbols = list(split_symbols(symbols, 2))
print(f"Splitting symbols into {len(chunked_symbols)} chunks.")

Splitting symbols into 2303 chunks.


In [25]:

# Download and store data for each chunk
for chunk in chunked_symbols[:2]:
    print(chunk)
    download_and_store(chunk)

['A', 'AA']
[*********************100%***********************]  2 of 2 completed
           Symbol       Close        High         Low        Open   Volume
Date                                                                      
2021-01-08      A  124.644684  125.213660  123.457686  124.085519  1577200
2021-01-11      A  125.713966  126.881348  123.830473  124.046285  1746800
2021-01-12      A  124.556412  125.949417  123.801045  125.890560  1826400
2021-01-13      A  124.566238  125.615898  124.330795  125.037110  1530400
2021-01-14      A  123.634277  125.753221  123.428270  124.850704   924900
...           ...         ...         ...         ...         ...      ...
2024-01-02     AA   33.310001   33.970001   33.040001   33.529999  3838000
2024-01-03     AA   31.490000   32.270000   31.045000   32.119999  8245700
2024-01-04     AA   31.129999   31.490000   30.860001   31.129999  4543600
2024-01-05     AA   32.130001   32.305000   30.090000   30.540001  7313000
2024-01-08     AA  

ValueError: Expected a 1D array, got an array with shape (1508, 754)